In [3]:
import pandas as pd
from google.colab import files

# Load the CSV file
df = pd.read_csv("Cleaned_File.csv")

# Show all column names (to check for typos or spaces)
print(df.columns.tolist())


['University', 'Category', 'Year Built', 'Description', 'Location', 'Phone', 'Email', 'Coordinator', 'Amenities', 'Detail Page URL', 'Program Category', 'Faculty', 'Program Name', 'Program Description', 'Years Of Study', 'Number Of Semesters', 'Fee In USD', 'Fee In EGP', 'Prerequisites', 'Credit Hours', 'Max Study Years', 'Affiliated Universities', 'Semesters Abroad']


In [4]:
# Fix and create the combined_text column with correct column names
df['combined_text'] = (
    "[University]: " + df['University'].fillna('') + "\n" +
    "[Program]: " + df['Program Name'].fillna('') + "\n" +
    "[Degree Description]: " + df['Program Description'].fillna('') + "\n" +
    "[Years of Study]: " + df['Years Of Study'].fillna('').astype(str) + "\n" +
    "[Fee in USD]: " + df['Fee In USD'].fillna('').astype(str) + "\n" +
    "[Location]: " + df['Location'].fillna('') + "\n" +
    "[More Info]: " + df['Detail Page URL'].fillna('')
)

# Preview a few rows to confirm
df[['University', 'Program Name', 'combined_text']].head()


,University,Program Name,combined_text
0,"Cairo Higher Institute for Languages, Interpre...",NaN,[University]: Cairo Higher Institute for Langu...
1,Ain Shams University ASU,Bachelor of Medicine,[University]: Ain Shams University ASU\n[Progr...
2,Ain Shams University ASU,Bachelor of Pharmaceutical sciences (Pharm D),[University]: Ain Shams University ASU\n[Progr...
3,Ain Shams University ASU,Bachelor of Pharmaceutical Sciences (Pharm D C...,[University]: Ain Shams University ASU\n[Progr...
4,Ain Shams University ASU,Oral and Dental Medicine,[University]: Ain Shams University ASU\n[Progr...


In [6]:
# Save updated file
df.to_csv("newest_QA_Passage_Data.csv", index=False)

# Download it
from google.colab import files
files.download("newest_QA_Passage_Data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# STEP 3: FAISS + SBERT Embedding and Indexing
!pip install -q faiss-cpu sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.6 MB/s eta 0:00:00


In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [10]:
# Initialize Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast & good for semantic search

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Convert combined_text to embeddings
corpus = df['combined_text'].fillna('').tolist()
embeddings = model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/550 [00:00<?, ?it/s]

In [12]:
# Convert embeddings to numpy array (float32 for FAISS)
embedding_matrix = np.array(embeddings).astype('float32')

In [13]:
# Build FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # L2 = Euclidean distance
index.add(embedding_matrix)

In [14]:
# Save index and mapping
faiss.write_index(index, "faiss_index.idx")
df[['combined_text']].to_csv("faiss_passages.csv", index=False)

print("✅ FAISS index built and saved.")

✅ FAISS index built and saved.


In [15]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load previously saved index and passages
index = faiss.read_index("faiss_index.idx")
passages_df = pd.read_csv("faiss_passages.csv")
model = SentenceTransformer('all-MiniLM-L6-v2')  # Same model used for indexing

In [16]:
# 🔍 Search Function
def search_faiss(query, top_k=3):
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    results = passages_df.iloc[indices[0]]
    return results

In [19]:
# ✨ Example Query
query = "What are the tuition fees for dentistry at Ain Shams University?"
results = search_faiss(query, top_k=10)

In [20]:
# Display Results
for i, row in results.iterrows():
    print(f"\n📄 Top Result #{i+1}:\n{row['combined_text']}\n{'='*80}")


📄 Top Result #1167:
[University]: Ain Shams University ASU
[Program]: Quality Management
[Degree Description]: New program - Extra tuition fees, a payment per credit hour per semester (minimum of 12 CH and maximum of 18 CH). Price of each hour is 174 EGP. In addition to the payment of 1500 dollars to be paid once.
[Years of Study]: 4.0
[Fee in USD]: 4000.0
[Location]: El-Khalifa El-Maamoun Street, El-Abbassia, 11566 Cairo Egypt
[More Info]: https://study-in-egypt.gov.eg/categories/All/8/Ain-Shams-University-ASU

📄 Top Result #1137:
[University]: Ain Shams University ASU
[Program]: Clinical Dentistry
[Degree Description]: Bachelor's degree of Clinical Dentistry (Credit Hours Program)
[Years of Study]: 5.0
[Fee in USD]: 6000.0
[Location]: El-Khalifa El-Maamoun Street, El-Abbassia, 11566 Cairo Egypt
[More Info]: https://study-in-egypt.gov.eg/categories/All/8/Ain-Shams-University-ASU

📄 Top Result #6:
[University]: Ain Shams University ASU
[Program]: Clinical Dentistry
[Degree Description

In [21]:
from transformers import pipeline

# Load Extractive QA pipeline
extractive_qa = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Take the top result from FAISS search
context = results.iloc[0]['combined_text']

# Clean the context to help BERT understand better
clean_context = context.replace("\n", " ").replace("  ", " ")

# Define the question clearly
question = "How much is the tuition fee in USD for dentistry at Ain Shams University?"

# Get the extractive answer
extractive_answer = extractive_qa(question=question, context=clean_context)
print("📌 Extractive Answer:", extractive_answer['answer'])


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


📌 Extractive Answer: 4.0


In [22]:
"question: <QUESTION>  context: <TOP-K PASSAGES>"


'question: <QUESTION>  context: <TOP-K PASSAGES>'

In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Combine the top 3 FAISS results as context
top_k_context = "\n".join(results['combined_text'].tolist())

# Refined prompt to guide T5 better
prompt = f"""
Given the following university program information, answer the question:

Context:
{top_k_context}

Question:
How much is the tuition fee for dentistry at Ain Shams University?
"""

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate output
outputs = model.generate(inputs['input_ids'], max_length=80, early_stopping=True)

# Decode and print
generative_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("💬 Generative Answer:", generative_answer)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


💬 Generative Answer: True


In [24]:
# Install and import necessary libraries
!pip install -q transformers accelerate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the smarter model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [25]:
# Prepare a clean version of your top FAISS results (filter by relevance)
filtered_passages = [p for p in results['combined_text'] if "dentist" in p.lower() or "fee" in p.lower()]
context_for_t5 = "\n".join(filtered_passages)

# Create a very clear prompt
prompt = f"""
Answer the following question based on the given university program information:

Question: What is the tuition fee for the dentistry program at Ain Shams University?

Information:
{context_for_t5}
"""

# Encode the prompt
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate the answer
outputs = model.generate(inputs['input_ids'], max_length=80, early_stopping=True)

# Decode and print
generative_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("💬 Generative Answer:", generative_answer)


💬 Generative Answer: More Info]: https://study-in-egypt.gov.eg/categories/All/8/Ain-Shams-University-ASU


In [1]:
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
outputs = generator.generate(inputs['input_ids'], max_length=80, early_stopping=True)
print("🔎 Generated:", tokenizer.decode(outputs[0], skip_special_tokens=True))


NameError: name 'prompt' is not defined